<a href="https://colab.research.google.com/github/nhut-ngnn/Practice-Graph_Neural_Network/blob/main/GNN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q torch

In [4]:
%%capture
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['PYTHONWARNINGS'] = "ignore"
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
!pip install torch-geometric  # Install PyTorch Geometric directly
from torch_geometric.datasets import Planetoid

In [7]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.
print(data)


Processing...


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Done!


In [8]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


In [14]:
model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()
      out = model(data.x, data.edge_index)
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

def test(mask):
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)
      correct = pred[mask] == data.y[mask]
      acc = int(correct.sum()) / int(mask.sum())
      return acc


val_acc_all = []
test_acc_all = []
for epoch in range(1, 101):
    loss = train()
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    val_acc_all.append(val_acc)
    test_acc_all.append(test_acc)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Loss: 1.9463, Val: 0.2880, Test: 0.2700
Epoch: 002, Loss: 1.9409, Val: 0.2580, Test: 0.2910
Epoch: 003, Loss: 1.9343, Val: 0.2560, Test: 0.2910
Epoch: 004, Loss: 1.9275, Val: 0.2640, Test: 0.3210
Epoch: 005, Loss: 1.9181, Val: 0.3220, Test: 0.3630
Epoch: 006, Loss: 1.9086, Val: 0.4000, Test: 0.4120
Epoch: 007, Loss: 1.9015, Val: 0.3900, Test: 0.4010
Epoch: 008, Loss: 1.8933, Val: 0.3900, Test: 0.4020
Epoch: 009, Loss: 1.8808, Val: 0.4220, Test: 0.4180
Epoch: 010, Loss: 1.8685, Val: 0.4560, Test: 0.4470
Epoch: 011, Loss: 1.8598, Val: 0.4760, Test: 0.4680
Epoch: 012, Loss: 1.8482, Val: 0.5120, Test: 0.5180
Epoch: 013, Loss: 1.8290, Val: 0.5380, Test: 0.5440
Epoch: 014, Loss: 1.8233, Val: 0.5580, Test: 0.5720
Epoch: 015, Loss: 1.8057, Val: 0.5820, Test: 0.5910
Epoch: 016, Loss: 1.7966, Val: 0.6060, Test: 0.6080
Epoch: 017, Loss: 1.7825, Val: 0.6200, Test: 0.6300
Epoch: 018, Loss: 1.7617, Val: 0.6280, Test: 0.6450
Epoch: 019, Loss: 1.7491, Val: 0.6280, Test: 0.6520
Epoch: 020, 